In [1]:
from tqdm import tqdm
from utils import *
import json
from openai import OpenAI
import numpy as np
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
load_dotenv()
client = OpenAI()
delete_cache()

Deleting: __pycache__
All __pycache__ directories have been deleted.


In [2]:
items = []
with open("train.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        items.append(json.loads(line))

In [3]:
embeddings = []
records = []  # 保留原始数据，方便回溯

for item in tqdm(items):
    question = item["question"].replace("\n", " ")
    think = item["think"].replace("\n", " ")

    text = f"問題：{question} 解析：{think} 答案：{item['answer']}"

    emb = client.embeddings.create(
        model="text-embedding-3-small",
        input=[text]
    ).data[0].embedding

    embeddings.append(emb)
    records.append(item)

embeddings = np.array(embeddings)


100%|██████████| 99/99 [00:39<00:00,  2.49it/s]


In [4]:
model_id = "Qwen/Qwen3-0.6B"  # 需提前下载/授权
tokenizer = AutoTokenizer.from_pretrained(
    model_id, 
    use_fast=False, 
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map="auto", 
    dtype=torch.bfloat16
)


'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 36029d78-9750-4911-a882-2a7464e4d118)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen3-0.6B/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 5b4fb2ba-a876-40fa-84a5-00ccd9531505)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen3-0.6B/resolve/main/tokenizer_config.json
Retrying in 2s [Retry 2/5].


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [5]:
def search(query, embeddings, chunks, top_k=5):
    query = query.replace("\n", " ")
    query_vec = client.embeddings.create(
        model="text-embedding-3-small",
        input=[query]
    ).data[0].embedding
    query_vec = np.array(query_vec)

    scores = np.dot(embeddings, query_vec)
    top_indices = np.argsort(scores)[-top_k:][::-1]
    return [chunks[i] for i in top_indices]

In [6]:
pred_list = []

with open("./val.jsonl", "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        line = line.strip()
        if not line:
            continue

        data = json.loads(line)
        question = data["question"]
        results = search(question, embeddings, records, 3)
        # print(results)
        context = "\n".join(
            f"問題：{r['question'].replace('\n', ' ')} \n"
            f"解析：{r['think'].replace('\n', ' ')} \n"
            f"答案：{r['answer'].replace('\n', ' ')} \n"
            for r in results
        )

        print()
        prompt = f"""
你是一个香港保险经纪人，你在考试，你需要正确回答考试题目。
你可以参考的信息：
{context}
回答格式举例（请注意，你的回答要由选项字母结尾，不要有多余的话）。
理由：...（简单说一下理由即可）
答案：A/B/C/D（所有题目都是单选题）
对于有一些题目的选项，例如“以上皆正确”或“以上皆不正确”，这种选项较难，但说不定是对的，请深思熟虑后做出选择。
"""
        print(f"prompt如下：{prompt}\n\n")
        messages = [
            {"role": "system", "content": prompt},
            {"role": "user", "content": question}
        ]

        response = predict(messages, model, tokenizer)

        # 取 response 最后一个字符（防止空字符串）
        last_char = response[-1] if response else None
        pred_list.append(last_char)
        print(response)
        print(f"{idx+1}: {last_char}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



prompt如下：
你是一个香港保险经纪人，你在考试，你需要正确回答考试题目。
你可以参考的信息：
問題：以下哪些不是危疾保險利益附約的陳述正確？ a)某些保險人可能選擇每月支付保險利益 b)危疾保障僅適用於某歲數或以下的人 c)受保事件發生後，受保保單所有人有時獲給付部分的死亡保險金 d)受保事件發生後，受保保單所有人總是須要繼續繳交保費 
解析：每月支付保險利益不屬危疾保險利益 附約的選擇，受保事件發生後，危疾 保險利益的受保保單所有人將獲給付 死亡保險金的載明部分，若所界定喪 失工作能力發生了三個月後，才會豁 免保費所有續保保費 
答案：A 

問題：以下哪項是關於保費抵銷的正確描述？ a)將來的現金紅利存放於保險人那裏 b)將來的紅利購買減額清繳保險 c)對未來業績表現有利 d)將來的紅利回報有利 
解析：保費抵銷預期將來的現金紅利存放於 保險人那裏，保單持有人有權選擇停 止繳付保費，但須注意是，未來利率 水平可以下降的，到時客戶須要 恢復掏錢付保費，否則可能導致保單 失效，只有答案A正確。 
答案：A 

問題：以下哪項有關遞減定期壽險的描述是不正確？ a)這是定期壽險其中一種變化 b)現有人壽保險種類中最便宜的 c)適合於逐漸減少的短暫保障需求 d)支付的利益按保單持有人的實際需要而變化 
解析：支付的利益按保單持有人的實際需要 而變化，不是遞減定期壽險的描述 
答案：D 

回答格式举例（请注意，你的回答要由选项字母结尾，不要有多余的话）。
理由：...（简单说一下理由即可）
答案：A/B/C/D（所有题目都是单选题）
对于有一些题目的选项，例如“以上皆正确”或“以上皆不正确”，这种选项较难，但说不定是对的，请深思熟虑后做出选择。



<think>
嗯，我现在要解决这个关于自然保費制度特点的问题。题目是问哪一项属于自然保費制度的特性。选项是a到d四个。

首先，我需要回忆一下自然保費制度的基本概念。自然保費，也就是自然保险，通常指的是那些保险人基于保单持有人的年龄、健康状况等因素来决定保费的。比如，年龄越大，保费越高，或者健康状况越差，保费越高。这应该是和年龄相关，而不是其他选项。

选项a说保费随年龄增长而增加。这听起来和自然保費一致，因为年龄越大，保险费用越高。所以a可能是正确选项。

选项b提到会引发逆选择。逆选择通常是指投保人为了保险而支

In [7]:
pred_list

['。',
 'D',
 'C',
 'B',
 'C',
 'C',
 'D',
 'D',
 'B',
 'C',
 'B',
 '。',
 '。',
 'A',
 '。',
 '。',
 'C',
 'C',
 'C',
 'D',
 'A',
 'D',
 'A',
 'A',
 '。']

In [8]:
correct = 0
total = 0

with open("./val.jsonl", "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        line = line.strip()
        if not line:
            continue

        data = json.loads(line)
        gold = data["answer"]  # 你已确认是 A / B / C / D

        pred = pred_list[idx]
        pred = pred.upper() if pred is not None else None

        total += 1
        if pred == gold:
            correct += 1

accuracy = correct / total if total > 0 else 0.0

print(f"Accuracy: {accuracy:.2f} ({correct}/{total})")

Accuracy: 0.56 (14/25)
